In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import re
import os
import subprocess
from pathlib import Path

SESSIONID = "CCFD30EDBE5EAB681A25EE87E6BD668E"

In [ ]:
courseid_001 = "63d3a60fe4b0f86ecf1be779"
courseid_002 = "63d3e0d3e4b09032e4cb167c"
COOKIES = {"SESSIONID":SESSIONID}

SELECTED_COURSE_ID = courseid_002

In [ ]:
r = requests.get(f"https://https://aathishsivasubrahmanian.graphy.com/s/mycourses?type=active&lsb",cookies=COOKIES)
apkid = re.search(r'apkId = \"(.*)\", theme9',r.text).group(1)
apkid

In [ ]:

r = requests.get(f"https://https://aathishsivasubrahmanian.graphy.com/s/courses/{SELECTED_COURSE_ID}/take",cookies=COOKIES)
soup = BeautifulSoup(r.content)
mydivs = soup.find("div",{"data-title":"Sample Course 1"}).find_all("div", {"class": "courseSubItem"})
viddict = {}
for div in mydivs:
    viddict[div.attrs["data-title"]] = div.attrs['data-id']

print(json.dumps(viddict,indent=4))

In [ ]:
# completedvids = ("01","02","03","04","05","06","07","08","09","10","11", "12",'13',"14","15","16","17","18","19","31","32","33","34","35","36","46", '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57')
incompletedvids =("12",'13',"14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29")

for vid in viddict:
    # if vid.startswith(completedvids) or isinstance(viddict[vid],dict):
    if not vid.startswith(incompletedvids):
        continue
    r = requests.get(f"https://https://aathishsivasubrahmanian.graphy.com/s/courses/{SELECTED_COURSE_ID}/videos/{viddict[vid]}/get?_=1699857984568",cookies=COOKIES)
    print(r.text)
    viddict[vid] = r.json()
print(json.dumps(viddict,indent=4))

In [ ]:
print(json.dumps(viddict,indent=4))

In [ ]:
for vid in viddict:
    if not isinstance(viddict[vid],dict):
        continue 
    try:
        viddirpath = Path(vid)
        os.mkdir(vid)
    except Exception as e:
        print(e)
        pass
    print(f"Starting Download : {vid}")
    url = viddict[vid]["spayee:resource"]["spayee:streamUrl"]
    r = requests.get(url,cookies=COOKIES)
    # with open(viddirpath/"index.m3u8","wb") as f:
    #     f.write(r.content)
    NEEDS_SEPARATE_AUDIO = ("hls_audio_.m3u8" in r.text)
    
    r = requests.get(url[:-10]+"hls_500k_.m3u8",cookies=COOKIES)
    with open(viddirpath/"hls_500k_.m3u8","w") as f:
        f.write(r.content.decode("utf-8").replace("k/timestamp","key_video.bin"))
    
    with open(viddirpath/"hls_500k_.m3u8","r") as f:
        with open(viddirpath/"key_video.bin","wb") as f2:
            r = requests.get(url[:-10]+"k/timestamp",cookies=COOKIES)
            timestampreturn = list(bytearray(r.content))
            bytestowrite = eval((subprocess.check_output(["node","decrypt_key.js", apkid, str(timestampreturn)])).decode("utf-8").strip())
            f2.write(bytes(bytestowrite))
    #     for line in f.readlines():
    #         if line.startswith("hls_500k"):
    #             r = requests.get(url[:-10]+line.strip(),cookies=COOKIES)
    #             with open(viddirpath/line.strip(),"wb") as f2:
    #                 f2.write(r.content)

    if NEEDS_SEPARATE_AUDIO:
        r = requests.get(url[:-10]+"hls_audio_.m3u8",cookies=COOKIES)
        with open(viddirpath/"hls_audio_.m3u8","w") as f:
            f.write(r.content.decode("utf-8").replace("k/timestamp","key_audio.bin"))
        with open(viddirpath/"hls_audio_.m3u8","r") as f:
            with open(viddirpath/"key_audio.bin","wb") as f2:
                r = requests.get(url[:-10]+"k/timestamp",cookies=COOKIES)
                timestampreturn = list(bytearray(r.content))
                bytestowrite = eval((subprocess.check_output(["node","decrypt_key.js", apkid, str(timestampreturn)])).decode("utf-8").strip())
                f2.write(bytes(bytestowrite))
        #     for line in f.readlines():
        #         if line.startswith("hls_audio"):
        #             r = requests.get(url[:-10]+line.strip(),cookies=COOKIES)
        #             with open(viddirpath/line.strip(),"wb") as f2:
        #                 f2.write(r.content)
    print(f"Finished : {vid}")
    # break


In [ ]:
/Users/aathishs/Projects/CTF/graphytty/ffmpeg-darwin-arm64 -allowed_extensions ALL -i '/Users/aathishs/Projects/CTF/graphytty/sample_video_1" -o "sample_video_1.mp4"

In [ ]:
import os
from pathlib import Path
curdir = Path(os.path.abspath(""))
folders = sorted([i for i in os.listdir() if i.startswith( ("1","2"))])
print(folders)
# input()
for folder in folders:
    os.system(f'/Users/aathishs/Projects/CTF/graphytty/ffmpeg-darwin-arm64 -allowed_extensions ALL -i "{folder}/index.m3u8" -c copy -vcodec copy "/Users/aathishs/Projects/CTF/graphytty/Videos/{folder}.mp4"')

